In [1]:
from datetime import datetime
import functools
from IPython.display import HTML
import jax
from jax import numpy as jp
import numpy as np
from typing import Any, Dict, Sequence, Tuple, Union
from brax import base
from brax import envs
from brax import math
from brax.base import Base, Motion, Transform
from brax.envs.base import Env, PipelineEnv, State
from brax.mjx.base import State as MjxState
# from brax.training.agents.apg import train as apg
# from brax.training.agents.apg import networks as apg_networks
from brax.io import html, mjcf, model
from etils import epath
from flax import struct
from matplotlib import pyplot as plt
import mediapy as media
from ml_collections import config_dict
import mujoco
from mujoco import mjx
from jax import vmap
import jax.random
from jax import lax

In [2]:
import distutils.util
import os
xla_flags = os.environ.get('XLA_FLAGS', '')
xla_flags += ' --xla_gpu_triton_gemm_any=True'
os.environ['XLA_FLAGS'] = xla_flags
print('Setting environment variable to use GPU rendering:')
%env MUJOCO_GL=egl

Setting environment variable to use GPU rendering:
env: MUJOCO_GL=egl


In [3]:
import os
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.8" # 0.9 causes too much lag. 
from datetime import datetime
import functools

# Math
import jax.numpy as jp
import numpy as np
import jax
from jax import config # Analytical gradients work much better with double precision.
config.update("jax_debug_nans", True)
config.update("jax_enable_x64", True)
config.update('jax_default_matmul_precision', jax.lax.Precision.HIGH)
from brax import math

# Sim
import mujoco
import mujoco.mjx as mjx

# Brax
from brax import envs
from brax.base import Motion, Transform
from brax.io import mjcf
from brax.envs.base import PipelineEnv, State
from brax.mjx.pipeline import _reformat_contact
from brax.training.acme import running_statistics
from brax.io import model

# Algorithms
# from brax.training.agents.apg import train as apg
# from brax.training.agents.apg import networks as apg_networks
from brax.training.agents.ppo import train as ppo

# Supporting
from etils import epath
import mediapy as media
import matplotlib.pyplot as plt
from ml_collections import config_dict
from typing import Any, Dict


In [4]:
from jax.lib import xla_bridge
print(xla_bridge.get_backend().platform)

gpu


In [7]:
import agent_mimic_env
from agent_mimic_env import register_mimic_env, register_ppo_env

In [8]:
from agent_mimic_env.pds_controllers_agents import feedback_pd_controller, stable_pd_controller_action


In [9]:
import yaml
from box import Box
# Path to your YAML file
yaml_file_path = 'config_params/punch.yaml'
# Load the YAML file
with open(yaml_file_path, 'r') as file:
    args = Box(yaml.safe_load(file))

print(args)

{'num_envs': 256, 'num_eval_envs': 32, 'lr': 0.0003, 'max_it': 1000, 'max_grad_norm': 0.4, 'seed': 0, 'system_config': 'humanoid', 'demo_replay_mode': 'threshold', 'threshold': 0.4, 'normalize_observations': True, 'cycle_len': 0, 'ep_len': 30, 'ep_len_eval': 54, 'use_lr_scheduler': True, 'reward_scaling': 0.02, 'rot_weight': 0.5, 'vel_weight': 0.01, 'ang_weight': 0.01, 'deep_mimic_reward_weights': {'w_p': 0.3, 'w_v': 0.1, 'w_e': 0.5, 'w_c': 0.1}, 'deep_mimic_weights_factors': {'w_pose': 2, 'w_angular': 0.005, 'w_efector': 5, 'w_com': 100}, 'ref': 'motions/humanoid3d_punch.txt'}


In [10]:
env_ppo = register_ppo_env(args)
env_ppo.set_pd_callback(stable_pd_controller_action)
jit_reset = jax.jit(env_ppo.reset)
jit_step= jax.jit(env_ppo.step)



In [11]:
print(env_ppo.sys.opt)
print(env_ppo.dt)

Option(timestep=0.002, impratio=Array(100., dtype=float64, weak_type=True), tolerance=Array(1.e-08, dtype=float64, weak_type=True), ls_tolerance=Array(0.01, dtype=float64, weak_type=True), gravity=Array([ 0.  ,  0.  , -9.81], dtype=float64), wind=Array([0., 0., 0.], dtype=float64), density=Array(0., dtype=float64, weak_type=True), viscosity=Array(0., dtype=float64, weak_type=True), has_fluid_params=False, integrator=<IntegratorType.EULER: 0>, cone=<ConeType.PYRAMIDAL: 0>, jacobian=<JacobianType.AUTO: 2>, solver=<SolverType.NEWTON: 2>, iterations=1, ls_iterations=6, disableflags=<DisableBit.EULERDAMP: 16384>)
0.016


In [12]:
state = jit_reset(jax.random.PRNGKey(0))
rollout = [state.pipeline_state]

n_steps = 500

In [13]:
print(state.metrics)
print(state.info)

{'fall': Array(0., dtype=float64), 'pose_error': Array(0., dtype=float64), 'reference_angular': Array(0., dtype=float64, weak_type=True), 'reference_com': Array(0., dtype=float64, weak_type=True), 'reference_end_effector': Array(0., dtype=float64, weak_type=True), 'reference_quaternions': Array(0., dtype=float64, weak_type=True), 'step_index': Array(0, dtype=int64, weak_type=True)}
{'reward_tuple': {'reference_angular': Array(0., dtype=float64, weak_type=True), 'reference_com': Array(0., dtype=float64, weak_type=True), 'reference_end_effector': Array(0., dtype=float64, weak_type=True), 'reference_quaternions': Array(0., dtype=float64, weak_type=True)}, 'rng': Array([0, 0], dtype=uint32), 'steps': Array(0., dtype=float64, weak_type=True)}


In [14]:
state.pipeline_state.subtree_com[1]

Array([0.08595606, 0.01209264, 0.80377791], dtype=float64)

In [15]:
env_ppo.rollout_lenght

65

In [21]:
state.pipeline_state.qpos[:28]



(28,)

In [20]:
for i in range(200):
    
    #print(i)
    ctrl = -0.1 * jp.ones(env_ppo.sys.nu)
    #time
    #time = state.pipeline_state.time
    
    print('time: ',state.pipeline_state.time)    
         
    
    state = jit_step(state,ctrl)
    
    #print(state.pipeline_state.geom_xpos)
    print("Is done", state.done)
    if state.done:
        print(state.pipeline_state.time)
        break
    
    #print("Is rewards", state.reward)
    # print("Is quat", state.metrics['reference_quaternions'])
    # print("Is ang",state.metrics['reference_angular'])
    # print("Is ee", state.metrics['reference_end_effector'])
    # print("Is com", state.metrics['reference_com'])

    #print("reward tuple", state.info['reward_tuple'])
    #print("Is ", state.metrics['pose_error'])
    #print("Is ", state.metrics['step_index'])
    
    rollout.append(state.pipeline_state)

time:  0.0
Is done 0.0
time:  0.016
Is done 0.0
time:  0.032000000000000015
Is done 0.0
time:  0.04800000000000003
Is done 0.0
time:  0.06400000000000004
Is done 0.0
time:  0.08000000000000006
Is done 0.0
time:  0.09600000000000007
Is done 0.0
time:  0.11200000000000009
Is done 0.0
time:  0.12800000000000009
Is done 0.0
time:  0.1440000000000001
Is done 0.0
time:  0.16000000000000011
Is done 0.0
time:  0.17600000000000013
Is done 0.0
time:  0.19200000000000014
Is done 0.0
time:  0.20800000000000016
Is done 0.0
time:  0.22400000000000017
Is done 0.0
time:  0.24000000000000019
Is done 0.0
time:  0.25600000000000017
Is done 0.0
time:  0.2720000000000002
Is done 0.0
time:  0.2880000000000002
Is done 0.0
time:  0.3040000000000002
Is done 0.0
time:  0.32000000000000023
Is done 0.0
time:  0.33600000000000024
Is done 0.0
time:  0.35200000000000026
Is done 0.0
time:  0.36800000000000027
Is done 0.0
time:  0.3840000000000003
Is done 0.0
time:  0.4000000000000003
Is done 0.0
time:  0.416000000000

In [18]:
HTML(html.render(env_ppo.sys.tree_replace({'opt.timestep': env_ppo.dt}), rollout))